# TODO:

- prepare xgboost model quickly for first sub DONE
- Set up kaggle kernel for submission (need whl for category_encoders and lifelines) DONE
- Set up Competition scoring DONE
- Improve on first model:
    - Set up CV, 
    - Tune hyper parameter 
    - try different preprocessing steps (No Change -> None, freq/None, freq/None is best) 
    - try stratification on race
- try other targets with xgboost
- try cox estimator
- try NN
- ensemble everything

In [49]:
%load_ext autoreload
%autoreload 2
import sys, os, glob
import gc
import time
import ast
from itertools import product, combinations, batched

import numpy as np
import pandas as pd
import pandas.api.types
from pandas.api.types import CategoricalDtype
import polars as pl
import polars.selectors as cs
import mlflow
import mlflow.xgboost

from sklearn.preprocessing import TargetEncoder, OneHotEncoder
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import validation_curve, ValidationCurveDisplay
import joblib
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

from category_encoders import CountEncoder
from category_encoders.hashing import HashingEncoder
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from tqdm.auto import tqdm

sys.path.insert(0, os.path.abspath('../scripts'))
from metric import score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
COMP_DATA_BASE = os.path.join("..", "data", "comp")
PREP_DATA_BASE = os.path.join("..", "data", "preprocessed")

TRAIN_PATH = os.path.join(COMP_DATA_BASE, "train.csv")
TEST_PATH = os.path.join(COMP_DATA_BASE, "test.csv")
DATA_DICT_PATH = os.path.join(COMP_DATA_BASE, "data_dictionary.csv")
SAMPLE_PATH = os.path.join(COMP_DATA_BASE, "sample_submission.csv")

features = list(pl.read_csv(TEST_PATH).select(pl.all().exclude("ID")).columns)
train_ds = pl.read_csv(TRAIN_PATH)

# Preprocessing

In [51]:
from data import prepare_data

train_prep_ds, test_prep_ds, feature_cols = prepare_data(pl.read_csv(TRAIN_PATH), pl.read_csv(DATA_DICT_PATH))
test_prep_ds

,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,...,hepatic_mild,tce_div_match,donor_related,melphalan_dose,cardiac,pulm_moderate,ID,efs,efs_time,target_cox
0,2.0,7.0,5.0,5.0,9.0,2.0,6.0,2.0,1.0,2.0,...,No,Permissive mismatched,Related,"N/A, Mel not given",Yes,Yes,18932,0.0,25.751,-25.751
1,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,No,NaN,Unrelated,NaN,No,No,21280,1.0,6.711,6.711
2,2.0,7.0,5.0,5.0,9.0,2.0,4.0,2.0,1.0,2.0,...,Yes,Permissive mismatched,Related,"N/A, Mel not given",Yes,Yes,27880,1.0,6.196,6.196
3,1.0,4.0,4.0,3.0,5.0,1.0,5.0,2.0,1.0,2.0,...,No,Permissive mismatched,Related,MEL,No,No,15692,1.0,3.204,3.204
4,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,No,Permissive mismatched,Related,"N/A, Mel not given",NaN,NaN,25416,1.0,3.283,3.283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2.0,8.0,6.0,6.0,NaN,NaN,6.0,2.0,2.0,NaN,...,No,NaN,Related,"N/A, Mel not given",No,No,27888,1.0,7.048,7.048
2876,2.0,7.0,5.0,5.0,9.0,2.0,5.0,1.0,2.0,2.0,...,No,Permissive mismatched,Unrelated,MEL,NaN,Yes,7456,0.0,33.922,-33.922
2877,2.0,7.0,6.0,5.0,9.0,2.0,6.0,2.0,2.0,2.0,...,No,NaN,Unrelated,MEL,No,No,14213,1.0,3.304,3.304
2878,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,No,GvH non-permissive,Unrelated,"N/A, Mel not given",No,No,25989,0.0,68.985,-68.985


# Feature Engineering

In [89]:
# 1. Groupby(featuesX)[featureY].mean() median  or std, min, max
# 2. Ratios between columns (/, +, *, ^)
# 3. Combining categorical features
# 4. diff() with mean and/or std
train_ds = pl.read_csv(TRAIN_PATH)

num_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Numerical"))
num_features = num_features_dict.select("variable").to_series().to_list()
cat_features_dict = pl.read_csv(DATA_DICT_PATH).filter(pl.col("variable").is_in(features) & (pl.col("type") == "Categorical"))
cat_features = cat_features_dict.select("variable").to_series().to_list()


# 1. Groupby(featuesX)[featureY].mean() median  or std, min, max
group_expr = [ [pl.col(num_feat).mean().over(cat_feat).alias(f"n_group_{cat_feat}_{num_feat}_mean"),
    pl.col(num_feat).median().over(cat_feat).alias(f"n_group_{cat_feat}_{num_feat}_median"),
    pl.col(num_feat).std().over(cat_feat).alias(f"n_group_{cat_feat}_{num_feat}_std"),
    pl.col(num_feat).min().over(cat_feat).alias(f"n_group_{cat_feat}_{num_feat}_min"),
    pl.col(num_feat).max().over(cat_feat).alias(f"n_group_{cat_feat}_{num_feat}_max")]
    for cat_feat, num_feat in product(cat_features, num_features) 
    ]
group_expr = [ exp for group in group_expr for exp in group]
print(f"New group cols: {len(group_expr)}")
# train_ds.with_columns(group_expr)

# 2. Ratios between columns (/, +, *, ^)
ratio_expr = [ [(pl.col(num_feat1) / pl.col(num_feat2)).replace([float("inf"), float("-inf")], None).clip(-1_000_000, 1_000_000).alias(f"n_ratio_{num_feat1}_{num_feat2}_div"),
    (pl.col(num_feat1) + pl.col(num_feat2)).replace([float("inf"), float("-inf")], None).clip(-1_000_000, 1_000_000).alias(f"n_ratio_{num_feat1}_{num_feat2}_add"),
    (pl.col(num_feat1) * pl.col(num_feat2)).replace([float("inf"), float("-inf")], None).clip(-1_000_000, 1_000_000).alias(f"n_ratio_{num_feat1}_{num_feat2}_prod"),
    (pl.col(num_feat1) ** pl.col(num_feat2)).replace([float("inf"), float("-inf")], None).clip(-1_000_000, 1_000_000).alias(f"n_ratio_{num_feat1}_{num_feat2}_pow")
    ]
    for num_feat1, num_feat2 in product(num_features, num_features) if num_feat1 !=  num_feat2
]
ratio_expr = [ exp for group in ratio_expr for exp in group]
print(f"New ratio cols: {len(ratio_expr)}")


# 3. Combining categorical features
comb_expr = [ [(pl.col(cat_feat1).fill_null("NULL") + "_" + pl.col(cat_feat2).fill_null("NULL")).alias(f"c_comb_{cat_feat1}_{cat_feat2}_concat"),
               pl.col(cat_feat1).mode().first().over(cat_feat2).alias(f"c_comb_{cat_feat1}_{cat_feat2}_mode"),

    ]
    for cat_feat1, cat_feat2 in combinations(cat_features, 2) 
]
comb_expr = [ exp for group in comb_expr for exp in group]
print(f"New ratio cols: {len(comb_expr)}")
# train_ds.with_columns(comb_expr)

# 4. diff() with mean and/or std
diff_expr = [ [(pl.col(num_feat).mean() - pl.col(num_feat)).alias(f"n_diff_{num_feat}_mean"),
               (pl.col(num_feat).median() - pl.col(num_feat)).alias(f"n_diff_{num_feat}_median"),
               (pl.col(num_feat).std() - pl.col(num_feat)).alias(f"n_diff_{num_feat}_std"),

    ]
    for num_feat in num_features
]
diff_expr = [ exp for group in diff_expr for exp in group]
print(f"New ratio cols: {len(diff_expr)}")
# train_ds.with_columns(comb_expr)


New group cols: 3850
New ratio cols: 1848
New ratio cols: 1190
New ratio cols: 66


# Modelling

## XGBoost

### Cox target

In [73]:
# cv

FOLDS = 5
SEED = 42

# Use for different hyperparameters, different preprocessing steps or feature selection/engineering
def evaluate_cv(model_config, train, test, target_col, features, folds, seed, refit=True):

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    cv_eval_xgb = 0
    with mlflow.start_run():
        eval_metric = 'cox-nloglik'
        model_xgb_cox = XGBRegressor(
            device="cuda",
            enable_categorical=True,
            objective='survival:cox',
            eval_metric=eval_metric,
            random_state=seed,
            **model_config
        )
        mlflow.log_params(model_config)
        mlflow.log_param("features", features)
        mlflow.log_param("target", target_col)
        
        for i, (train_index, test_index) in enumerate(kf.split(train)):
            # print("#"*25)
            # print(f"### Fold {i+1}")
            # print("#"*25)
            
            x_train = train.loc[train_index, features].copy()
            y_train = train.loc[train_index,target_col]    
            x_valid = train.loc[test_index, features].copy()
            y_valid = train.loc[test_index,target_col]
            x_test = test[features].copy()
                
            model_xgb_cox.fit(
                x_train, y_train,
                eval_set=[(x_valid, y_valid)],  
                verbose=500  
            )
            
            cv_eval_xgb += model_xgb_cox.evals_result()['validation_0'][eval_metric][-1]
            
            # INFER OOF
            oof_xgb[test_index] = model_xgb_cox.predict(x_valid)
            # INFER TEST
            pred_xgb += model_xgb_cox.predict(x_test)
        
        # COMPUTE AVERAGE TEST PREDS
        pred_xgb /= FOLDS
        cv_eval_xgb /= FOLDS
        mlflow.log_metric(f"{eval_metric} CV ", cv_eval_xgb)
        
        # Re-fit on whole data
        if refit:
            if target not in test.columns:
                model_xgb_cox.fit(
                    train[features], train[target_col],
                    eval_set=[(train[features], train[target_col])],  
                    verbose=500  
                )
            else:
                model_xgb_cox.fit(
                    train[features], train[target_col],
                    eval_set=[(train[features], train[target_col]), (test[features], test[target_col])],  
                    verbose=500  
                )
            
            eval_results = model_xgb_cox.evals_result()
            mlflow.xgboost.log_model(model_xgb_cox, "model")
            
            pred_xgb_refit = model_xgb_cox.predict(train[features])
            pred_xgb_test_refit = model_xgb_cox.predict(test[features])
        
        
        # Compute and log metrics
        
        y_true = train[["ID","efs","efs_time","race_group"]].copy()
        y_pred = train[["ID"]].copy()
        y_pred["prediction"] = oof_xgb
        m_cv = score(y_true.copy(), y_pred.copy(), "ID")
        print(f"\nOverall CV for XGBoost Survival:Cox =",m_cv)
        mlflow.log_metric("Comp Metric CV", m_cv)
        
        if refit:
            y_true = train[["ID","efs","efs_time","race_group"]].copy()
            y_pred = train[["ID"]].copy()
            y_pred["prediction"] = pred_xgb_refit
            m = score(y_true.copy(), y_pred.copy(), "ID")
            # print(f"\nOverall for refut XGBoost Survival:Cox =",m)
            mlflow.log_metric("Comp Metric Refit train", m)
            
            score_train = model_xgb_cox.score(train[features], train[target_col])
            mlflow.log_metric("R squared Refit train ", score_train)
            
            xgb_eval_train = eval_results['validation_0'][eval_metric][-1]
            mlflow.log_metric(f"{eval_metric} Refit train ", xgb_eval_train)
        
        
        if target in test.columns:
            y_true = test[["ID","efs","efs_time","race_group"]].copy()
            y_pred = test[["ID"]].copy()
            y_pred["prediction"] = pred_xgb
            m = score(y_true.copy(), y_pred.copy(), "ID")
            mlflow.log_metric("Comp Metric CV test", m)
            # print(f"\nOverall CV for XGBoost Survival:Cox =",m)
            
            if refit:
                y_true = test[["ID","efs","efs_time","race_group"]].copy()
                y_pred = test[["ID"]].copy()
                y_pred["prediction"] = pred_xgb_test_refit
                m = score(y_true.copy(), y_pred.copy(), "ID")
                mlflow.log_metric("Comp Metric Refit test", m)
                
                score_test = model_xgb_cox.score(test[features], test[target_col])
                mlflow.log_metric("R squared Refit test ", score_test)
                
                xgb_eval_test = eval_results['validation_1'][eval_metric][-1]
                mlflow.log_metric(f"{eval_metric} Refit train ", xgb_eval_test)

        
        
        
    return m_cv, pred_xgb, oof_xgb


# Test method:                                 
target = "target_cox"
train_prep_ds, test_prep_ds, feature_cols =  prepare_data(pl.read_csv(TRAIN_PATH), pl.read_csv(DATA_DICT_PATH), target_col=target)

# .evals_result()
config_default = {"max_depth": 3,  
            "colsample_bytree":0.5,  
            "subsample":0.8,  
            "n_estimators":500,  
            "learning_rate":0.02,  
            "min_child_weight":80}
results = evaluate_cv(config_default, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED, refit=False)


[0]	validation_0-cox-nloglik:8.20574
[499]	validation_0-cox-nloglik:8.01165
[0]	validation_0-cox-nloglik:8.20517
[499]	validation_0-cox-nloglik:8.01698
[0]	validation_0-cox-nloglik:8.20870
[499]	validation_0-cox-nloglik:8.01438
[0]	validation_0-cox-nloglik:8.20177
[499]	validation_0-cox-nloglik:8.01575
[0]	validation_0-cox-nloglik:8.20590
[499]	validation_0-cox-nloglik:8.01098


c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



Overall CV for XGBoost Survival:Cox = 0.6592182321431516


c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)


In [90]:
# Feature selection
BATCH_SIZE = 25
expr = group_expr + ratio_expr + comb_expr + diff_expr
print(f"List of size {len(expr)} with {len(list(batched( expr , n=BATCH_SIZE)))} batches of size {BATCH_SIZE} ")

# 0.6637377405131833
config = {'alpha': 0.014196756627406019, 'colsample_bytree': 0.8515695986963434, 'gamma': 0.20749935045175233, 'learning_rate': 0.031138095926462342, 'max_leaves': 183, 'min_child_weight': 8, 'n_estimators': 453, 'subsample': 0.6333008368148032}

# Initial features
train_initial = pl.read_csv(TRAIN_PATH)
data_dict_initial = pl.read_csv(DATA_DICT_PATH)
train_prep_ds, test_prep_ds, feature_cols =  prepare_data(train_initial, data_dict_initial , target_col=target)

target = "target_cox"
results = evaluate_cv(config, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=5, seed=42)
current_metric = results[0]

good_batches = []


for batch in tqdm(batched( reversed(expr) , n=BATCH_SIZE)):

    train_next = train_initial.with_columns(list(batch))

    new_cols_type = {col: ("Numerical" if col[0] == 'n' else "Categorical") for col in train_next.columns if col not in train_initial.columns}
    new_rows = [{"variable": col, "description": "", "type": new_cols_type[col], "values": ""} for col in new_cols_type.keys()]
    new_df = pl.DataFrame(new_rows)                           
    data_dict_next = pl.concat([data_dict_initial, new_df]) 

    train_prep_ds, test_prep_ds, feature_cols =  prepare_data(train_next, data_dict_next , target_col=target)
    print(train_next.shape, train_next.columns)
    print( train_prep_ds.shape, list(train_prep_ds.columns), "\n")
    results = evaluate_cv(config, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=5, seed=42, refit=False)
    
    # keep features if metric is better
    if (results[0] - current_metric) > 0.0001:
        current_metric = results[0]
        train_initial = train_next
        data_dict_initial = data_dict_next
        
        good_batches += batch
        
    


List of size 6954 with 279 batches of size 25 
[0]	validation_0-cox-nloglik:8.20049
[452]	validation_0-cox-nloglik:8.01272
[0]	validation_0-cox-nloglik:8.20016
[452]	validation_0-cox-nloglik:8.02620
[0]	validation_0-cox-nloglik:8.20367
[452]	validation_0-cox-nloglik:8.00592
[0]	validation_0-cox-nloglik:8.19679
[452]	validation_0-cox-nloglik:8.02321
[0]	validation_0-cox-nloglik:8.20100
[452]	validation_0-cox-nloglik:8.00565
[0]	validation_0-cox-nloglik:9.80847	validation_1-cox-nloglik:7.61996
[452]	validation_0-cox-nloglik:9.34071	validation_1-cox-nloglik:7.41130


c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:44:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/02/01 19:45:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Marc\AppData\Local\Temp\tmpvn7g7kj4\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/02/01 19:45:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Marc\OneDrive\projects\Kaggl


Overall CV for XGBoost Survival:Cox = 0.6676958104384855


c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and

0it [00:00, ?it/s]

...

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)


(28800, 164) ['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10', 'efs', 'efs_time', 'c_comb_vent_hist_donor_r

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



Overall CV for XGBoost Survival:Cox = 0.6691459095379719


c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)


In [104]:
train_ds.with_columns(good_batches).select(pl.all().exclude(train_ds.columns)).columns

['c_comb_vent_hist_donor_related_concat',
 'c_comb_vent_hist_tce_div_match_mode',
 'c_comb_vent_hist_tce_div_match_concat',
 'c_comb_vent_hist_hepatic_mild_mode',
 'c_comb_vent_hist_hepatic_mild_concat',
 'c_comb_vent_hist_race_group_mode',
 'c_comb_vent_hist_race_group_concat',
 'c_comb_vent_hist_sex_match_mode',
 'c_comb_vent_hist_sex_match_concat',
 'c_comb_vent_hist_rheum_issue_mode',
 'c_comb_vent_hist_rheum_issue_concat',
 'c_comb_vent_hist_gvhd_proph_mode',
 'c_comb_vent_hist_gvhd_proph_concat',
 'c_comb_vent_hist_peptic_ulcer_mode',
 'c_comb_vent_hist_peptic_ulcer_concat',
 'c_comb_vent_hist_prior_tumor_mode',
 'c_comb_vent_hist_prior_tumor_concat',
 'c_comb_vent_hist_hepatic_severe_mode',
 'c_comb_vent_hist_hepatic_severe_concat',
 'c_comb_vent_hist_tce_match_mode',
 'c_comb_vent_hist_tce_match_concat',
 'c_comb_vent_hist_in_vivo_tcd_mode',
 'c_comb_vent_hist_in_vivo_tcd_concat',
 'c_comb_vent_hist_mrd_hct_mode',
 'c_comb_vent_hist_mrd_hct_concat',
 'n_ratio_hla_match_a_high_h

In [ ]:
features = ['c_comb_vent_hist_donor_related_concat',
 'c_comb_vent_hist_tce_div_match_mode',
 'c_comb_vent_hist_tce_div_match_concat',
 'c_comb_vent_hist_hepatic_mild_mode',
 'c_comb_vent_hist_hepatic_mild_concat',
 'c_comb_vent_hist_race_group_mode',
 'c_comb_vent_hist_race_group_concat',
 'c_comb_vent_hist_sex_match_mode',
 'c_comb_vent_hist_sex_match_concat',
 'c_comb_vent_hist_rheum_issue_mode',
 'c_comb_vent_hist_rheum_issue_concat',
 'c_comb_vent_hist_gvhd_proph_mode',
 'c_comb_vent_hist_gvhd_proph_concat',
 'c_comb_vent_hist_peptic_ulcer_mode',
 'c_comb_vent_hist_peptic_ulcer_concat',
 'c_comb_vent_hist_prior_tumor_mode',
 'c_comb_vent_hist_prior_tumor_concat',
 'c_comb_vent_hist_hepatic_severe_mode',
 'c_comb_vent_hist_hepatic_severe_concat',
 'c_comb_vent_hist_tce_match_mode',
 'c_comb_vent_hist_tce_match_concat',
 'c_comb_vent_hist_in_vivo_tcd_mode',
 'c_comb_vent_hist_in_vivo_tcd_concat',
 'c_comb_vent_hist_mrd_hct_mode',
 'c_comb_vent_hist_mrd_hct_concat',
 'n_ratio_hla_match_a_high_hla_high_res_8_div',
 'n_ratio_hla_match_a_high_hla_match_c_high_pow',
 'n_ratio_hla_match_a_high_hla_match_c_high_prod',
 'n_ratio_hla_match_a_high_hla_match_c_high_add',
 'n_ratio_hla_match_a_high_hla_match_c_high_div',
 'n_ratio_year_hct_hla_low_res_10_pow',
 'n_ratio_year_hct_hla_low_res_10_prod',
 'n_ratio_year_hct_hla_low_res_10_add',
 'n_ratio_year_hct_hla_low_res_10_div',
 'n_ratio_year_hct_hla_match_drb1_high_pow',
 'n_ratio_year_hct_hla_match_drb1_high_prod',
 'n_ratio_year_hct_hla_match_drb1_high_add',
 'n_ratio_year_hct_hla_match_drb1_high_div',
 'n_ratio_year_hct_hla_low_res_8_pow',
 'n_ratio_year_hct_hla_low_res_8_prod',
 'n_ratio_year_hct_hla_low_res_8_add',
 'n_ratio_year_hct_hla_low_res_8_div',
 'n_ratio_year_hct_karnofsky_score_pow',
 'n_ratio_year_hct_karnofsky_score_prod',
 'n_ratio_year_hct_karnofsky_score_add',
 'n_ratio_year_hct_karnofsky_score_div',
 'n_ratio_year_hct_comorbidity_score_pow',
 'n_ratio_year_hct_comorbidity_score_prod',
 'n_ratio_year_hct_comorbidity_score_add',
 'n_ratio_year_hct_comorbidity_score_div',
 'n_group_prim_disease_hct_hla_match_dqb1_high_min',
 'n_group_prim_disease_hct_hla_match_dqb1_high_std',
 'n_group_prim_disease_hct_hla_match_dqb1_high_median',
 'n_group_prim_disease_hct_hla_match_dqb1_high_mean',
 'n_group_prim_disease_hct_hla_high_res_10_max',
 'n_group_prim_disease_hct_hla_high_res_10_min',
 'n_group_prim_disease_hct_hla_high_res_10_std',
 'n_group_prim_disease_hct_hla_high_res_10_median',
 'n_group_prim_disease_hct_hla_high_res_10_mean',
 'n_group_prim_disease_hct_hla_high_res_6_max',
 'n_group_prim_disease_hct_hla_high_res_6_min',
 'n_group_prim_disease_hct_hla_high_res_6_std',
 'n_group_prim_disease_hct_hla_high_res_6_median',
 'n_group_prim_disease_hct_hla_high_res_6_mean',
 'n_group_prim_disease_hct_hla_low_res_6_max',
 'n_group_prim_disease_hct_hla_low_res_6_min',
 'n_group_prim_disease_hct_hla_low_res_6_std',
 'n_group_prim_disease_hct_hla_low_res_6_median',
 'n_group_prim_disease_hct_hla_low_res_6_mean',
 'n_group_prim_disease_hct_hla_high_res_8_max',
 'n_group_prim_disease_hct_hla_high_res_8_min',
 'n_group_prim_disease_hct_hla_high_res_8_std',
 'n_group_prim_disease_hct_hla_high_res_8_median',
 'n_group_prim_disease_hct_hla_high_res_8_mean',
 'n_group_prim_disease_hct_hla_match_c_high_max',
 'n_group_renal_issue_hla_match_dqb1_low_min',
 'n_group_renal_issue_hla_match_dqb1_low_std',
 'n_group_renal_issue_hla_match_dqb1_low_median',
 'n_group_renal_issue_hla_match_dqb1_low_mean',
 'n_group_renal_issue_hla_match_drb1_low_max',
 'n_group_renal_issue_hla_match_drb1_low_min',
 'n_group_renal_issue_hla_match_drb1_low_std',
 'n_group_renal_issue_hla_match_drb1_low_median',
 'n_group_renal_issue_hla_match_drb1_low_mean',
 'n_group_renal_issue_hla_match_c_low_max',
 'n_group_renal_issue_hla_match_c_low_min',
 'n_group_renal_issue_hla_match_c_low_std',
 'n_group_renal_issue_hla_match_c_low_median',
 'n_group_renal_issue_hla_match_c_low_mean',
 'n_group_renal_issue_hla_nmdp_6_max',
 'n_group_renal_issue_hla_nmdp_6_min',
 'n_group_renal_issue_hla_nmdp_6_std',
 'n_group_renal_issue_hla_nmdp_6_median',
 'n_group_renal_issue_hla_nmdp_6_mean',
 'n_group_renal_issue_hla_match_dqb1_high_max',
 'n_group_renal_issue_hla_match_dqb1_high_min',
 'n_group_renal_issue_hla_match_dqb1_high_std',
 'n_group_renal_issue_hla_match_dqb1_high_median',
 'n_group_renal_issue_hla_match_dqb1_high_mean',
 'n_group_renal_issue_hla_high_res_10_max']

In [97]:
train_ds.with_columns(expr)

TypeError: the truth value of an Expr is ambiguous

You probably got here by using a Python standard library function instead of the native expressions API.
Here are some things you might want to try:
- instead of `pl.col('a') and pl.col('b')`, use `pl.col('a') & pl.col('b')`
- instead of `pl.col('a') in [y, z]`, use `pl.col('a').is_in([y, z])`
- instead of `max(pl.col('a'), pl.col('b'))`, use `pl.max_horizontal(pl.col('a'), pl.col('b'))`


In [9]:
# 1. Choose target type
#   ["target1", "target2", "target_km", "target_cox"]
# 2. Choose preprocesing options
#    - Add numeric features with few values as categorical
# 3. Hyperparameter Tuning
#   - Try first to get some senseful values before step 1 and 2
# 4. Feature Enigneering/Selection 
#     - Groupby(featuesX)[featureY].mean()

target = "target_cox"
passthrough_cols = ["ID","efs","efs_time","race_group", target]
train_prep_ds, test_prep_ds, feature_cols = preprocess(train_data, test_data, target, passthrough_cols, numeric_type="robust", low_cardinality_type="ohe",high_cardinality_type="hashing" , impute=False)

# define an objective function
def objective(args):
    int_vals = ["n_estimators", "max_leaves", "max_depth", "min_child_weight"]
    space = {k: (int(val) if k in int_vals else val) for k, val in args.items()}
    
    # scale_pos_weight = sum(negative instances) / sum(positive instances)
    # if not "scale_pos_weight" in space: 
    #     space["scale_pos_weight"]  = 5.5882
    FOLDS = 5
    SEED = 42

    results = evaluate_cv(space, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED)

    print("Run result:", results[0])
    return {'loss': -results[0], 'status': STATUS_OK }

# define a search space
space = {
    'n_estimators': hp.quniform('n_estimators',100,500, 1),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.2),
    'max_leaves': hp.quniform('max_leaves', 10, 200,1),
    # 'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_weight':  hp.quniform('min_child_weight', 0, 10,1),
}



trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=50, trials=trials)

print(best)
print(space_eval(space, best)) 

[0]	validation_0-cox-nloglik:8.17046                  
[443]	validation_0-cox-nloglik:8.10756                
[0]	validation_0-cox-nloglik:8.17108                  
[443]	validation_0-cox-nloglik:8.11332                
[0]	validation_0-cox-nloglik:8.17558                  
[443]	validation_0-cox-nloglik:8.09314                
[0]	validation_0-cox-nloglik:8.16778                  
[443]	validation_0-cox-nloglik:8.15039                
[0]	validation_0-cox-nloglik:8.17109                  
[443]	validation_0-cox-nloglik:8.07766                
                                                      
Overall CV for XGBoost Survival:Cox =
0.6556620233558825                                    
Run result:                                           
0.6556620233558825                                    
[0]	validation_0-cox-nloglik:8.18742                                             
[471]	validation_0-cox-nloglik:8.01147                                           
[0]	validation_0-cox-nloglik

In [ ]:
config = {'alpha': 0.014196756627406019, 'colsample_bytree': 0.8515695986963434, 'gamma': 0.20749935045175233, 'learning_rate': 0.031138095926462342, 'max_leaves': 183, 'min_child_weight': 8, 'n_estimators': 453, 'subsample': 0.6333008368148032}

all_preprocessor_types = [{"numeric_type": type} for type in [None, "quantile", "robust", "minmax", "standard"] ] \
                        + [{"low_cardinality_type": type} for type in [None, "target", "ohe", "frequency"] ]  \
                        + [{"high_cardinality_type": type} for type in [None, "target", "frequency", "ohe", "hashing"] ] 

target = "target_cox"
passthrough_cols = ["ID","efs","efs_time","race_group", target]

               
# 100 types
prep_type_iter = product([{"numeric_type": type} for type in [None, "quantile", "robust", "minmax", "standard"] ], 
        [{"low_cardinality_type": type} for type in [None, "target", "ohe", "frequency"] ], 
        [{"high_cardinality_type": type} for type in [None, "target", "frequency", "ohe", "hashing"]])
best_run = None
best_run_score = 0

search_results = pd.DataFrame({"num_type":[] , "low_card_type":[], "high_card_type":[], "score": []})
for num_type, low_card_type, high_card_type in tqdm(prep_type_iter):
    train_prep_ds, test_prep_ds, feature_cols = preprocess(train_data, test_data, target, passthrough_cols, **num_type, **low_card_type, **high_card_type, impute=False)
    results = evaluate_cv(config, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED)
    next_result = pd.DataFrame({"num_type":[num_type["numeric_type"]] , "low_card_type":[low_card_type["low_cardinality_type"]], "high_card_type":[high_card_type["high_cardinality_type"]], "score": [results[0]]})
    search_results = pd.concat([search_results, next_result], axis=0)
    print("Run result:", results[0])
    if results[0] > best_run_score:
        best_run = (num_type, low_card_type, high_card_type)
        best_run_score = results[0]        

print(best_run_score)
print(best_run)
display(search_results.sort_values(by="score", ascending=False))



In [20]:
print(search_results.sort_values(by="score", ascending=False).head(50))

   num_type low_card_type high_card_type     score
0  quantile     frequency           None  0.667875
0      None     frequency           None  0.667875
0    minmax     frequency           None  0.667875
0    robust     frequency           None  0.667875
0  standard     frequency           None  0.667875
0    robust          None      frequency  0.667609
0  standard          None      frequency  0.667609
0    minmax          None      frequency  0.667609
0      None          None      frequency  0.667609
0  quantile          None      frequency  0.667609
0      None          None           None  0.667408
0  quantile          None           None  0.667408
0    robust          None           None  0.667408
0    minmax          None           None  0.667408
0  standard          None           None  0.667408
0  quantile          None         target  0.667020
0  standard          None         target  0.667020
0    robust          None         target  0.667020
0    minmax          None      

### XGBOOST with kaplan maier

In [6]:
# cv

FOLDS = 5
SEED = 42
config = {"max_depth": 3,  
            "colsample_bytree":0.5,  
            "subsample":0.8,  
            "n_estimators":2000,  
            "learning_rate":0.02,  
            "min_child_weight":80}

# Use for different hyperparameters, different preprocessing steps or feature selection/engineering
def evaluate_cv(model_config, train, test, target_col, features, folds, seed):

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        # print("#"*25)
        # print(f"### Fold {i+1}")
        # print("#"*25)
        
        x_train = train.loc[train_index, features].copy()
        y_train = train.loc[train_index,target_col]    
        x_valid = train.loc[test_index, features].copy()
        y_valid = train.loc[test_index,target_col]
        x_test = test[features].copy()
            
        model_xgb_cox = XGBRegressor(
            device="cuda",
            enable_categorical=True,
            objective='reg:logistic',
            eval_metric='mae',
            **model_config
        )
        model_xgb_cox.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],  
            verbose=500  
        )
        # INFER OOF
        oof_xgb[test_index] = model_xgb_cox.predict(x_valid)
        # INFER TEST
        pred_xgb += model_xgb_cox.predict(x_test)
        
    # COMPUTE AVERAGE TEST PREDS
    pred_xgb /= FOLDS
    
    y_true = train[["ID","efs","efs_time","race_group"]].copy()
    y_pred = train[["ID"]].copy()
    y_pred["prediction"] = oof_xgb
    m = score(y_true.copy(), y_pred.copy(), "ID")
    print(f"\nOverall CV for XGBoost Kaplan Meier =",m)
    
    return m, pred_xgb, oof_xgb



In [7]:

target = "target_km"
train_prep_ds, test_prep_ds, feature_cols = train_prep_ds, test_prep_ds, feature_cols = prepare_data(pl.read_csv(TRAIN_PATH), pl.read_csv(DATA_DICT_PATH), target_col=target)

# define an objective function
def objective(args):
    int_vals = ["n_estimators", "max_leaves", "max_depth", "min_child_weight"]
    space = {k: (int(val) if k in int_vals else val) for k, val in args.items()}
    
    # scale_pos_weight = sum(negative instances) / sum(positive instances)
    # if not "scale_pos_weight" in space: 
    #     space["scale_pos_weight"]  = 5.5882
    FOLDS = 5
    SEED = 42

    results = evaluate_cv(space, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED)

    print("Run result:", results[0])
    return {'loss': -results[0], 'status': STATUS_OK }

# define a search space
space = {
    'n_estimators': hp.quniform('n_estimators',100,500, 1),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.2),
    'max_leaves': hp.quniform('max_leaves', 10, 200,1),
    # 'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_weight':  hp.quniform('min_child_weight', 0, 10,1),
}


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

print(best)
print(space_eval(space, best)) 

[0]	validation_0-mae:0.15548                           
[246]	validation_0-mae:0.13295                         
[0]	validation_0-mae:0.15424                           
  0%|          | 0/100 [00:01<?, ?trial/s, best loss=?]

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:47:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)



[246]	validation_0-mae:0.13294                         
[0]	validation_0-mae:0.15330                           
[246]	validation_0-mae:0.13139                         
[0]	validation_0-mae:0.15372                           
[246]	validation_0-mae:0.13232                         
[0]	validation_0-mae:0.15476                           
[246]	validation_0-mae:0.13280                         
  0%|          | 0/100 [00:06<?, ?trial/s, best loss=?]

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                       
Overall CV for XGBoost Kaplan Meier =
0.6625595747723556                                     
Run result:                                            
0.6625595747723556                                     
[0]	validation_0-mae:0.15554                                                      
[390]	validation_0-mae:0.13196                                                    
[0]	validation_0-mae:0.15432                                                      
[390]	validation_0-mae:0.13126                                                    
[0]	validation_0-mae:0.15338                                                      
[390]	validation_0-mae:0.12990                                                    
[0]	validation_0-mae:0.15385                                                      
[390]	validation_0-mae:0.13073                                                    
[0]	validation_0-mae:0.15481                                              

...

                                                                                   
Overall CV for XGBoost Kaplan Meier =
0.664691883837503                                                                  
Run result:                                                                        
0.664691883837503                                                                  
[0]	validation_0-mae:0.15567                                                       
[320]	validation_0-mae:0.13103                                                     
[0]	validation_0-mae:0.15444                                                       
[320]	validation_0-mae:0.13095                                                     
[0]	validation_0-mae:0.15349                                                       
[320]	validation_0-mae:0.12903                                                     
[0]	validation_0-mae:0.15390                                                       
[320]	validation_0-mae:0.13041        

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                                                   
Overall CV for XGBoost Kaplan Meier =
0.6660854521218864                                                                 
Run result:                                                                        
0.6660854521218864                                                                 
[0]	validation_0-mae:0.15483                                                       
[365]	validation_0-mae:0.13057                                                     
[0]	validation_0-mae:0.15356                                                       
[365]	validation_0-mae:0.13017                                                     
[0]	validation_0-mae:0.15265                                                       
[365]	validation_0-mae:0.12830                                                     
[0]	validation_0-mae:0.15306                                                       
[365]	validation_0-mae:0.12957        

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                                                   
Overall CV for XGBoost Kaplan Meier =
0.6653785854135544                                                                 
Run result:                                                                        
0.6653785854135544                                                                 
100%|██████████| 100/100 [18:23<00:00, 11.03s/trial, best loss: -0.6688989176771293]
{'alpha': np.float64(0.5616697545730821), 'colsample_bytree': np.float64(0.8768121695437451), 'gamma': np.float64(0.29248294164676425), 'learning_rate': np.float64(0.04975918457179248), 'max_leaves': np.float64(43.0), 'min_child_weight': np.float64(10.0), 'n_estimators': np.float64(479.0), 'subsample': np.float64(0.8063976736719021)}
{'alpha': 0.5616697545730821, 'colsample_bytree': 0.8768121695437451, 'gamma': 0.29248294164676425, 'learning_rate': 0.04975918457179248, 'max_leaves': 43.0, 'min_child_weight': 10.0, 'n_estimators': 479.0

In [ ]:
# 0.6653785854135544
config = {'alpha': 0.5616697545730821, 'colsample_bytree': 0.8768121695437451, 'gamma': 0.29248294164676425, 'learning_rate': 0.04975918457179248, 'max_leaves': 43, 'min_child_weight': 10, 'n_estimators': 479, 'subsample': 0.8063976736719021}

### XGBOOST with custom target 1

In [16]:
# cv

FOLDS = 5
SEED = 42

# Use for different hyperparameters, different preprocessing steps or feature selection/engineering
def evaluate_cv(model_config, train, test, target_col, features, folds, seed):

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        # print("#"*25)
        # print(f"### Fold {i+1}")
        # print("#"*25)
        
        x_train = train.loc[train_index, features].copy()
        y_train = train.loc[train_index,target_col]    
        x_valid = train.loc[test_index, features].copy()
        y_valid = train.loc[test_index,target_col]
        x_test = test[features].copy()
            
        model_xgb_cox = XGBRegressor(
            device="cuda",
            enable_categorical=True,
            objective='reg:logistic',
            eval_metric='mae',
            **model_config
        )
        model_xgb_cox.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],  
            verbose=500  
        )
        # INFER OOF
        oof_xgb[test_index] = model_xgb_cox.predict(x_valid)
        # INFER TEST
        pred_xgb += model_xgb_cox.predict(x_test)
        
    # COMPUTE AVERAGE TEST PREDS
    pred_xgb /= FOLDS
    
    y_true = train[["ID","efs","efs_time","race_group"]].copy()
    y_pred = train[["ID"]].copy()
    y_pred["prediction"] = oof_xgb
    m = score(y_true.copy(), y_pred.copy(), "ID")
    print(f"\nOverall CV for XGBoost Custom Target 1 =",m)
    
    return m, pred_xgb, oof_xgb



In [ ]:

target = "target1"
train_prep_ds, test_prep_ds, feature_cols = train_prep_ds, test_prep_ds, feature_cols = prepare_data(pl.read_csv(TRAIN_PATH), pl.read_csv(DATA_DICT_PATH), target_col=target)

# define an objective function
def objective(args):
    int_vals = ["n_estimators", "max_leaves", "max_depth", "min_child_weight"]
    space = {k: (int(val) if k in int_vals else val) for k, val in args.items()}
    
    # scale_pos_weight = sum(negative instances) / sum(positive instances)
    # if not "scale_pos_weight" in space: 
    #     space["scale_pos_weight"]  = 5.5882
    FOLDS = 5
    SEED = 42

    results = evaluate_cv(space, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED)

    print("Run result:", results[0])
    return {'loss': -results[0], 'status': STATUS_OK }

# define a search space
space = {
    'n_estimators': hp.quniform('n_estimators',100,1000, 1),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.2),
    'max_leaves': hp.quniform('max_leaves', 10, 200,1),
    # 'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_weight':  hp.quniform('min_child_weight', 0, 10,1),
}


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

print(best)
print(space_eval(space, best)) 

### XGBOOST with custom target 2

In [12]:
# cv

FOLDS = 5
SEED = 42

# Use for different hyperparameters, different preprocessing steps or feature selection/engineering
def evaluate_cv(model_config, train, test, target_col, features, folds, seed):

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    oof_xgb = np.zeros(len(train))
    pred_xgb = np.zeros(len(test))
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        # print("#"*25)
        # print(f"### Fold {i+1}")
        # print("#"*25)
        
        x_train = train.loc[train_index, features].copy()
        y_train = train.loc[train_index,target_col]    
        x_valid = train.loc[test_index, features].copy()
        y_valid = train.loc[test_index,target_col]
        x_test = test[features].copy()
            
        model_xgb_cox = XGBRegressor(
            device="cuda",
            enable_categorical=True,
            objective='reg:squarederror',
            eval_metric='mae',
            **model_config
        )
        model_xgb_cox.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],  
            verbose=500  
        )
        # INFER OOF
        oof_xgb[test_index] = model_xgb_cox.predict(x_valid)
        # INFER TEST
        pred_xgb += model_xgb_cox.predict(x_test)
        
    # COMPUTE AVERAGE TEST PREDS
    pred_xgb /= FOLDS
    
    y_true = train[["ID","efs","efs_time","race_group"]].copy()
    y_pred = train[["ID"]].copy()
    y_pred["prediction"] = oof_xgb
    m = score(y_true.copy(), y_pred.copy(), "ID")
    print(f"\nOverall CV for XGBoost Custom Target 2 =",m)
    
    return m, pred_xgb, oof_xgb


target = "target2"
train_prep_ds, test_prep_ds, feature_cols = train_prep_ds, test_prep_ds, feature_cols = prepare_data(pl.read_csv(TRAIN_PATH), pl.read_csv(DATA_DICT_PATH), target_col=target)

# define an objective function
def objective(args):
    int_vals = ["n_estimators", "max_leaves", "max_depth", "min_child_weight"]
    space = {k: (int(val) if k in int_vals else val) for k, val in args.items()}
    
    # scale_pos_weight = sum(negative instances) / sum(positive instances)
    # if not "scale_pos_weight" in space: 
    #     space["scale_pos_weight"]  = 5.5882
    FOLDS = 5
    SEED = 42

    results = evaluate_cv(space, train_prep_ds, test_prep_ds, target_col=target, features=feature_cols, folds=FOLDS, seed=SEED)

    print("Run result:", results[0])
    return {'loss': -results[0], 'status': STATUS_OK }

# define a search space
space = {
    'n_estimators': hp.quniform('n_estimators',100,500, 1),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.2),
    'max_leaves': hp.quniform('max_leaves', 10, 200,1),
    # 'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'gamma': hp.uniform('gamma', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_weight':  hp.quniform('min_child_weight', 0, 10,1),
}


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

print(best)
print(space_eval(space, best)) 

[0]	validation_0-mae:1.30808                           
[465]	validation_0-mae:1.09890                         
[0]	validation_0-mae:1.29734                           
[465]	validation_0-mae:1.09346                         
[0]	validation_0-mae:1.29490                           
[465]	validation_0-mae:1.08953                         
[0]	validation_0-mae:1.28953                           
[465]	validation_0-mae:1.10297                         
[0]	validation_0-mae:1.31011                           
[465]	validation_0-mae:1.10032                         
  0%|          | 0/100 [00:16<?, ?trial/s, best loss=?]

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                       
Overall CV for XGBoost Custom Target 2 =
0.661483490500217                                      
Run result:                                            
0.661483490500217                                      
[0]	validation_0-mae:1.30681                                                     
[412]	validation_0-mae:1.09765                                                   
[0]	validation_0-mae:1.29611                                                     
[412]	validation_0-mae:1.09604                                                   
[0]	validation_0-mae:1.29357                                                     
[412]	validation_0-mae:1.08893                                                   
[0]	validation_0-mae:1.28816                                                     
[412]	validation_0-mae:1.10610                                                   
[0]	validation_0-mae:1.30943                                                   

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                                                 
Overall CV for XGBoost Custom Target 2 =
0.6641327640125778                                                               
Run result:                                                                      
0.6641327640125778                                                               
[0]	validation_0-mae:1.29872                                                      
[228]	validation_0-mae:1.10325                                                    
[0]	validation_0-mae:1.28973                                                      
[228]	validation_0-mae:1.09669                                                    
[0]	validation_0-mae:1.28671                                                      
[228]	validation_0-mae:1.08302                                                    
[0]	validation_0-mae:1.28181                                                      
[228]	validation_0-mae:1.09965                    

...

                                                                                  
Overall CV for XGBoost Custom Target 2 =
0.664639656010437                                                                 
Run result:                                                                       
0.664639656010437                                                                 
[0]	validation_0-mae:1.30854                                                      
[446]	validation_0-mae:1.09349                                                    
[0]	validation_0-mae:1.29796                                                      
[446]	validation_0-mae:1.09013                                                    
[0]	validation_0-mae:1.29500                                                      
[446]	validation_0-mae:1.09060                                                    
[0]	validation_0-mae:1.29031                                                      
[446]	validation_0-mae:1.10488                

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                                                  
Overall CV for XGBoost Custom Target 2 =
0.6665797648232594                                                                
Run result:                                                                       
0.6665797648232594                                                                
[0]	validation_0-mae:1.31183                                                      
[409]	validation_0-mae:1.10077                                                    
[0]	validation_0-mae:1.30164                                                      
[409]	validation_0-mae:1.09661                                                    
[0]	validation_0-mae:1.29888                                                      
[409]	validation_0-mae:1.08850                                                    
[0]	validation_0-mae:1.29349                                                      
[409]	validation_0-mae:1.10336                

c:\Users\Marc\OneDrive\projects\Kaggle\CIBMTR_Equity_in_post_HCT_Survival_Predictions\scripts\metric.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)



                                                                                  
Overall CV for XGBoost Custom Target 2 =
0.6681495007196492                                                                
Run result:                                                                       
0.6681495007196492                                                                
100%|██████████| 100/100 [20:36<00:00, 12.36s/trial, best loss: -0.669226301685229]
{'alpha': np.float64(0.4298531906636763), 'colsample_bytree': np.float64(0.6310625983933004), 'gamma': np.float64(0.4898585146550601), 'learning_rate': np.float64(0.03726128890415703), 'max_leaves': np.float64(81.0), 'min_child_weight': np.float64(9.0), 'n_estimators': np.float64(459.0), 'subsample': np.float64(0.7856397050796257)}
{'alpha': 0.4298531906636763, 'colsample_bytree': 0.6310625983933004, 'gamma': 0.4898585146550601, 'learning_rate': 0.03726128890415703, 'max_leaves': 81.0, 'min_child_weight': 9.0, 'n_estimators': 459.0, 'sub

In [ ]:
# 0.6681495007196492
config = {'alpha': 0.4298531906636763, 'colsample_bytree': 0.6310625983933004, 'gamma': 0.4898585146550601, 'learning_rate': 0.03726128890415703, 'max_leaves': 81, 'min_child_weight': 9, 'n_estimators': 459, 'subsample': 0.7856397050796257}

In [13]:
print(best)
print(space_eval(space, best)) 

{'alpha': np.float64(0.4298531906636763), 'colsample_bytree': np.float64(0.6310625983933004), 'gamma': np.float64(0.4898585146550601), 'learning_rate': np.float64(0.03726128890415703), 'max_leaves': np.float64(81.0), 'min_child_weight': np.float64(9.0), 'n_estimators': np.float64(459.0), 'subsample': np.float64(0.7856397050796257)}
{'alpha': 0.4298531906636763, 'colsample_bytree': 0.6310625983933004, 'gamma': 0.4898585146550601, 'learning_rate': 0.03726128890415703, 'max_leaves': 81.0, 'min_child_weight': 9.0, 'n_estimators': 459.0, 'subsample': 0.7856397050796257}


In [15]:
trials['result']

NotImplementedError: 

## Other Models

## Ensemble and prepare submission